# chapter5 복수 가중치 동시에 학습하기
---

## 복수 입력을 받는 경사하강법

4장에서는 가중치를 갱신하기 위해 경사하강법을 사용하는 방법을 배웠다.  
이번 장에서는 복수 가중치가 있는 신경망을 갱신할 때 사용할 수 있는 기법을 알아보자.

In [41]:
#복수 입력을 받는 신경망
def w_sum(a, b):
    assert(len(a) == len(b))
    output = 0
    for i in range(len(a)):
        output += (a[i] * b[i])
    return output

#a(input)와 b(weights)의 각 attribute끼리 곱한 값의 총합을 output으로 돌려줌

In [66]:
weights = [0.1, 0.2, -.1]
def neural_network(input, weights):
    pred = w_sum(input, weights)
    return pred

#output을 pred로 돌려줌

In [43]:
#예측 + 비교: 예측을 수행하고 오차와 델타를 계산하기
toes = [8.5, 9.5, 9.9, 9.0] #발가락 개수
wlrec = [0.65, 0.8, 0.8, 0.9] #승패
nfans = [1.2, 1.3, 0.5, 1.0] #관중 수

win_or_lose_binary = [1, 1, 0, 1] #승리 1, 패배 0

true = win_or_lose_binary[0] 

input = [toes[0], wlrec[0], nfans[0]] #input data를 [8.5, 0.65, 1.2]

pred = neural_network(input, weights) #[8.5, 0.65, 1.2] . [0.1, 0.2, -.1]

error = (pred - true) ** 2 #평균제곱오차

delta = pred - true

print("error:", error)
print('delta:', delta)

error: 0.01959999999999997
delta: -0.1399999999999999


In [44]:
#학습: 각 weight_delta를 계산해서 각 가중치에 대입하기
def ele_mul(number, vector):
    output = [0, 0, 0]
    assert(len(output) == len(vector))
    for i in range(len(vector)):
        output[i] = number * vector[i]
    return output

weights_deltas = ele_mul(delta, input)

#delta에 input의 attribute를 곱한 값을 output으로 반환
#output = weights_deltas

In [67]:
#학습: 가중치 갱신하기
'''
신경망이 학습하는 속도를 제어하기 위함. 너무 빠르게 학습하면 weight를 공격적
으로 갱신해서 초과 예측(overshoot)을 할 수도 있기 때문.
'''
alpha = 0.01 #학습 전에 고정시킴
for i in range(len(weights)):
    weights[i] -= alpha * weights_deltas[i]
    
print("Weights:" + str(weights))
print("Weight Deltas:" + str(weights_deltas))

Weights:[0.1119, 0.20091, -0.09832]
Weight Deltas:[-1.189999999999999, -0.09099999999999994, -0.16799999999999987]


## 복수 입력을 받는 경사하강법 이해하기

In [ ]:
#단일 입력: 예측을 수행하고 오차와 델타를 계산하기
number_of_toes = [8.5]
win_or_lose_binary = [1]
input = number_of_toes[0]
true = win_or_lose_binary[0]
pred = neural_network(input, weight)
error = (pred - true) ** 2
delta = pred - true

In [ ]:
#복수 입력: 예측을 수행하고 오차와 델타를 계산하기
toes = [8.5, 9.5, 9.9, 9.0]
wlrec = [0.65, 0.8, 0.8, 0.9]
nfans = [1.2, 1.3, 0.5, 1.0]
win_or_lose_binary = [1, 1, 0, 1]
true = win_or_lose_binary[0]
input = [toes[0], wlrec[0], nfans[0]]
pred = neural_network(input, weights)
error = (pred - true) ** 2
delta = pred - true

단일 입력에서는 weight가 하나면 input도 하나였다. 하지만 weight가 3개일 때, weight_delta는 어떻게 만들까?

하나의 delta를 어떻게 3개의 weight_delta로 바꿀 수 있는가? delta는 공유하지만 input은 개별적이다. 즉, input에 delta를 곱한 값을 사용하면 저마다의 weight_delta를 얻을 수 있다.

delta란, 현재 주어진 학습 예제를 완벽하게 예측하기 위해 노드의 값이 얼마나 높아져야 하는지 또는 낮아져야 하는지를 나타내는 척도이다.  
weight delta란, 스케일링, 음의 반전, 종료를 처리. 노드에서의 delta를 감소시키기 위해 움직여야하는 가중치의 이동 방향과 거리에 대한 미분계수 기반 추정치이다.

스케일링: 순오차에 input을 곱하면서 생김. input이 크면 weight도 커짐. 통제할 수 없는 경우가 많아 alpha라는 것을 사용  
음의 반전: input이 음수일 때 순오차에 input을 곱하면 부호가 뒤집히게 하는 것  
종료: input이 0이면 direction_and_amount(weight를 어떻게 수정할지)도 0이 되도록 강제하는 것

## 학습의 각 단계를 관찰해보세요

In [84]:
def neural_network(input, weights):
  out = 0
  for i in range(len(input)):
    out += (input[i] * weights[i])
  return out

def ele_mul(scalar, vector):
  out = [0,0,0]
  for i in range(len(out)):
    out[i] = vector[i] * scalar
  return out

toes =  [8.5, 9.5, 9.9, 9.0]
wlrec = [0.65, 0.8, 0.8, 0.9]
nfans = [1.2, 1.3, 0.5, 1.0]

win_or_lose_binary = [1, 1, 0, 1]
true = win_or_lose_binary[0]

alpha = 0.01
weights = [0.1, 0.2, -.1]
input = [toes[0],wlrec[0],nfans[0]]

for iter in range(3):

  pred = neural_network(input,weights)

  error = (pred - true) ** 2
  delta = pred - true

  weight_deltas=ele_mul(delta,input)

  print("Iteration:" + str(iter+1))
  print("Pred:" + str(pred))
  print("Error:" + str(error))
  print("Delta:" + str(delta))
  print("Weights:" + str(weights))
  print("Weight_Deltas:")
  print(str(weight_deltas))
  print(
  )

  for i in range(len(weights)):
    weights[i]-=alpha*weight_deltas[i]

Iteration:1
Pred:0.8600000000000001
Error:0.01959999999999997
Delta:-0.1399999999999999
Weights:[0.1, 0.2, -0.1]
Weight_Deltas:
[-1.189999999999999, -0.09099999999999994, -0.16799999999999987]

Iteration:2
Pred:0.9637574999999999
Error:0.0013135188062500048
Delta:-0.036242500000000066
Weights:[0.1119, 0.20091, -0.09832]
Weight_Deltas:
[-0.30806125000000056, -0.023557625000000044, -0.04349100000000008]

Iteration:3
Pred:0.9906177228125002
Error:8.802712522307997e-05
Delta:-0.009382277187499843
Weights:[0.11498061250000001, 0.20114557625, -0.09788509000000001]
Weight_Deltas:
[-0.07974935609374867, -0.006098480171874899, -0.011258732624999811]



p.125~126의 그래프에서 곡선 a가 다른 곡선에 비해 더 가파른 이유는 input 값이 크기 때문이다.  
4차원 도형의 2차원 단면으로, 차원 3개는 weight 값이고, 네 번째 차원은 error이다. 이 도형을 오차 평면이라 부르고, 곡률은 학습 데이터가 결정한다.

입력이 기울기를 크게 수정하므로 가중치를 수정하는 학습은 대부분 큰 input을 받는 가중치에 대해서만 이뤄진다.정규화는 이와 같은 데이터셋의 한계를 극복하고 모든 가중치에 대해 학습이 이루어지도록 도와준다.

## 가중치 한 개 동결시키기

이는 각 가중치가 서로에게 어떤 영향을 미치는지 이해할 수 있는 훌륭한 연습이다. 가중치 0번째를 제외한 1, 2번째만을 활용하여 학습해보자.

In [87]:
def neural_network(input, weights):
  out = 0
  for i in range(len(input)):
    out += (input[i] * weights[i])
  return out

def ele_mul(scalar, vector):
  out = [0,0,0]
  for i in range(len(out)):
    out[i] = vector[i] * scalar
  return out

toes =  [8.5, 9.5, 9.9, 9.0]
wlrec = [0.65, 0.8, 0.8, 0.9]
nfans = [1.2, 1.3, 0.5, 1.0]

win_or_lose_binary = [1, 1, 0, 1]
true = win_or_lose_binary[0]

alpha = 0.01
weights = [0.1, 0.2, -.1]
input = [toes[0],wlrec[0],nfans[0]]

for iter in range(3):

  pred = neural_network(input,weights)

  error = (pred - true) ** 2
  delta = pred - true

  weight_deltas=ele_mul(delta,input)
  weight_deltas[0] = 0

  print("Iteration:" + str(iter+1))
  print("Pred:" + str(pred))
  print("Error:" + str(error))
  print("Delta:" + str(delta))
  print("Weights:" + str(weights))
  print("Weight_Deltas:")
  print(str(weight_deltas))
  print(
  )

  for i in range(len(weights)):
    weights[i]-=alpha*weight_deltas[i]

Iteration:1
Pred:0.8600000000000001
Error:0.01959999999999997
Delta:-0.1399999999999999
Weights:[0.1, 0.2, -0.1]
Weight_Deltas:
[0, -0.09099999999999994, -0.16799999999999987]

Iteration:2
Pred:0.8626075000000001
Error:0.018876699056249977
Delta:-0.13739249999999992
Weights:[0.1, 0.20091, -0.09832]
Weight_Deltas:
[0, -0.08930512499999994, -0.1648709999999999]

Iteration:3
Pred:0.8651664353125001
Error:0.018180090166338207
Delta:-0.13483356468749985
Weights:[0.1, 0.20180305125, -0.09667129]
Weight_Deltas:
[0, -0.0876418170468749, -0.16180027762499982]



이렇듯 weight_deltas[0]을 동결시켰음에도 불구하고 a의 검은 점의 error는 0을 향하는 것을 볼 수 있다.

우리가 신경망을 통해 궁극적으로 하고자 하는 것은 오차 평면에서 가장 낮은 점을 찾는 것이다.